### 上接泰坦尼克号探索一
#### 此次运用新的数据进行预测

In [1]:
# # 导出处理好的数据
# Xtrain.to_csv('Xtrain_good.csv',encoding='utf-8',index=False) #默认导出到当前路径，若有指定路径也可以
# Xtest.to_csv('Xtest_good.csv',encoding='utf-8',index=False)
# Ytrain.to_csv('Ytrain_good.csv',encoding='utf-8',index=False)
# Ytest.to_csv('Ytest_good.csv',encoding='utf-8',index=False)

# 一、导入库

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#style
plt.style.use('ggplot')



import warnings
warnings.filterwarnings('ignore')

# 二、导入数据

In [3]:
Xtrain=pd.read_csv('Xtrain_good.csv',encoding='utf-8')

In [4]:
Xtest=pd.read_csv('Xtest_good.csv',encoding='utf-8')

In [5]:
Ytrain=pd.read_csv('Ytrain_good.csv',encoding='utf-8')

In [6]:
Ytest=pd.read_csv('Ytest_good.csv',encoding='utf-8')

In [7]:
Xtrain.shape

(712, 8)

In [8]:
Xtest.shape

(179, 8)

In [9]:
Ytrain.shape

(712, 1)

In [10]:
Ytest.shape

(179, 1)

In [11]:
Xtrain.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked,result,Age_band
0,3,1,0,0,7.2292,0,2,2
1,2,0,0,0,10.5000,2,1,2
2,3,0,1,0,16.1000,2,3,3
3,1,1,0,0,0.0000,2,2,3
4,3,1,0,0,8.6625,2,2,1


***导入新数据***

In [12]:
test=pd.read_csv('test.csv')

In [13]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [14]:
# 查看test的基本信息
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [15]:
# 查看缺失情况
test.isnull().sum()
test.isnull().mean()

PassengerId    0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.205742
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.002392
Cabin          0.782297
Embarked       0.000000
dtype: float64

### 针对缺失的部分进行处理

- ***对缺失的年龄进行填补***

In [16]:
test['initial']=test.Name.str.extract('([A-Za-z]+)\.')
test['initial']

0          Mr
1         Mrs
2          Mr
3          Mr
4         Mrs
5          Mr
6        Miss
7          Mr
8         Mrs
9          Mr
10         Mr
11         Mr
12        Mrs
13         Mr
14        Mrs
15        Mrs
16         Mr
17         Mr
18       Miss
19        Mrs
20         Mr
21     Master
22        Mrs
23         Mr
24        Mrs
25         Mr
26       Miss
27         Mr
28         Mr
29         Mr
        ...  
388        Mr
389    Master
390        Mr
391       Mrs
392    Master
393        Mr
394        Mr
395       Mrs
396        Mr
397       Mrs
398        Mr
399        Mr
400      Miss
401        Mr
402      Miss
403        Mr
404        Mr
405        Mr
406        Mr
407        Mr
408      Miss
409      Miss
410      Miss
411       Mrs
412      Miss
413        Mr
414      Dona
415        Mr
416        Mr
417    Master
Name: initial, Length: 418, dtype: object

In [17]:
pd.crosstab(test.initial,test.Sex).T

initial,Col,Dona,Dr,Master,Miss,Mr,Mrs,Ms,Rev
Sex,,,,,,,,,
female,0,1,0,0,78,0,72,1,0
male,2,0,1,21,0,240,0,0,2


In [18]:
# 运用循环将小的合并成一类
result=[]
for i in test.initial.values:
    if i!= 'Master' and i!='Miss' and i!='Mr' and i!='Mrs':
        result.append(i.replace(i,'other'))
    else:
        result.append(i)

In [19]:
test['result']=result
test.result.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'other'], dtype=object)

In [20]:
test.groupby('result')['Sex'].count()

result
Master     21
Miss       78
Mr        240
Mrs        72
other       7
Name: Sex, dtype: int64

In [21]:
# 查看不同姓氏下的年龄的平均值
# 查看第一姓氏的年龄均值
# 选取整数
int(test.groupby('result')['Age'].mean()[0])

7

In [22]:
# 运用循环得到所有姓氏的年龄均值的整数用于填补空缺值
for i in test.result.values:
    test.loc[(test.Age.isnull())&(test.result==i),'Age']=int(test.groupby('result')['Age'].mean()[i])

In [23]:
# 查看是否还存在空缺值
test.Age.isnull().sum()

0

In [24]:
# 查看一次空缺的信息
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
initial          0
result           0
dtype: int64

In [25]:
# 年龄填补空缺后，接下来要对其进行划分区间即分箱
# 查看年龄的情况
test.Age.values

array([34.5 , 47.  , 62.  , 27.  , 22.  , 14.  , 30.  , 26.  , 18.  ,
       21.  , 32.  , 46.  , 23.  , 63.  , 47.  , 24.  , 35.  , 21.  ,
       27.  , 45.  , 55.  ,  9.  , 38.  , 21.  , 48.  , 50.  , 22.  ,
       22.5 , 41.  , 32.  , 50.  , 24.  , 33.  , 38.  , 30.  , 18.5 ,
       21.  , 21.  , 25.  , 32.  , 39.  , 32.  , 41.  , 30.  , 45.  ,
       25.  , 45.  , 32.  , 60.  , 36.  , 24.  , 27.  , 20.  , 28.  ,
       32.  , 10.  , 35.  , 25.  , 32.  , 36.  , 17.  , 32.  , 18.  ,
       22.  , 13.  , 38.  , 18.  , 47.  , 31.  , 60.  , 24.  , 21.  ,
       29.  , 28.5 , 35.  , 32.5 , 32.  , 55.  , 30.  , 24.  ,  6.  ,
       67.  , 49.  , 32.  , 32.  , 32.  , 27.  , 18.  , 43.  ,  2.  ,
       22.  , 32.  , 27.  , 32.  , 25.  , 25.  , 76.  , 29.  , 20.  ,
       33.  , 43.  , 27.  , 32.  , 26.  , 16.  , 28.  , 21.  , 32.  ,
       32.  , 18.5 , 41.  , 21.  , 36.  , 18.5 , 63.  , 18.  , 32.  ,
        1.  , 36.  , 29.  , 12.  , 32.  , 35.  , 28.  , 32.  , 17.  ,
       22.  , 21.  ,

In [26]:
# 分箱
test['Age_band']=0 #初始化年龄区间的人数值为0
test.loc[test['Age']<=18,'Age_band']=1
test.loc[(test['Age']>18)&(test['Age']<=30),'Age_band']=2
test.loc[(test['Age']>30)&(test['Age']<=50),'Age_band']=3
test.loc[(test['Age']>50),'Age_band']=4
test.Age_band

0      3
1      3
2      4
3      2
4      2
5      1
6      2
7      2
8      1
9      2
10     3
11     3
12     2
13     4
14     3
15     2
16     3
17     2
18     2
19     3
20     4
21     1
22     3
23     2
24     3
25     3
26     2
27     2
28     3
29     3
      ..
388    2
389    1
390    2
391    4
392    1
393    3
394    2
395    1
396    2
397    3
398    2
399    3
400    2
401    3
402    2
403    1
404    3
405    2
406    2
407    3
408    2
409    1
410    2
411    3
412    2
413    3
414    3
415    3
416    3
417    1
Name: Age_band, Length: 418, dtype: int64

In [29]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,initial,result,Age_band
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr,Mr,3
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Mrs,Mrs,3
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Mr,Mr,4
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Mr,Mr,2
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Mrs,Mrs,2


- **针对文本型数据进行编码，便于后续的建模**

In [34]:
# 导入预处理的包
from sklearn import preprocessing

In [37]:
# LabelEncoder 针对不连续数据或文本数据进行编码，将其变成连续的数值型的变量
# fit_transform 直接进行转化
lb=preprocessing.LabelEncoder()
test['Sex']=lb.fit_transform(test['Sex'])
test['Embarked']=lb.fit_transform(test['Embarked'])
test['result']=lb.fit_transform(test['result'])

In [38]:
# 查看Embarked编码后的结果
test.Embarked.unique()

array([1, 2, 0], dtype=int64)

In [47]:
test.Sex.value_counts()

1    266
0    152
Name: Sex, dtype: int64

In [48]:
# 查看性别编码后的结果
test['Sex'].value_counts()

1    266
0    152
Name: Sex, dtype: int64

In [51]:
test['result'].value_counts()

2    240
1     78
3     72
0     21
4      7
Name: result, dtype: int64

### 删除无关的列

In [53]:
test.drop(['PassengerId','Name','Age','Ticket','Cabin','initial'],axis=1)

,Pclass,Sex,SibSp,Parch,Fare,Embarked,result,Age_band
0,3,1,0,0,7.8292,1,2,3
1,3,0,1,0,7.0000,2,3,3
2,2,1,0,0,9.6875,1,2,4
3,3,1,0,0,8.6625,2,2,2
4,3,0,1,1,12.2875,2,3,2
5,3,1,0,0,9.2250,2,2,1
6,3,0,0,0,7.6292,1,1,2
7,2,1,1,1,29.0000,2,2,2
8,3,0,0,0,7.2292,0,3,1
9,3,1,2,0,24.1500,2,2,2


In [54]:
test.drop(['PassengerId','Name','Age','Ticket','Cabin','initial'],axis=1,inplace=True)

In [55]:
# 查看test
test.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked,result,Age_band
0,3,1,0,0,7.8292,1,2,3
1,3,0,1,0,7.0000,2,3,3
2,2,1,0,0,9.6875,1,2,4
3,3,1,0,0,8.6625,2,2,2
4,3,0,1,1,12.2875,2,3,2


In [56]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
Pclass      418 non-null int64
Sex         418 non-null int32
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
Embarked    418 non-null int32
result      418 non-null int32
Age_band    418 non-null int64
dtypes: float64(1), int32(3), int64(4)
memory usage: 21.3 KB


In [60]:
# Fare存在一个缺失值
test.Fare.isnull().sum()

1

In [59]:
test.Fare.describe()

count    417.000000
mean      35.627188
std       55.907576
min        0.000000
25%        7.895800
50%       14.454200
75%       31.500000
max      512.329200
Name: Fare, dtype: float64

- 由于数据的极差很大，表示存在异常值，因此选用中位数来填补空缺值

In [65]:
test.Fare[test.Fare.isnull()]=test.Fare.describe()[5]

In [67]:
test.Fare.isnull().sum()
# 填补完成

0

In [68]:
test.isnull().mean()

Pclass      0.0
Sex         0.0
SibSp       0.0
Parch       0.0
Fare        0.0
Embarked    0.0
result      0.0
Age_band    0.0
dtype: float64

鉴于探索一中的逻辑回归模型和通过网格搜索得到的最佳组合参数值的决策树模型的训练效果不错，此次也针对这两个模型进行一下新数据的预测

# 三、建立模型

In [69]:
from sklearn.linear_model import LogisticRegression #逻辑回归包
from sklearn.tree import DecisionTreeClassifier #决策树

- ***逻辑回归***

In [70]:
logist=LogisticRegression()
logist.fit(Xtrain,Ytrain)
logist.score(Xtest,Ytest)

0.8156424581005587

In [79]:
# 预测
test_pre=logist.predict(test)
test_pre

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [101]:
len(test_pre)

418

- ***决策树***

In [83]:
from sklearn.tree import DecisionTreeClassifier

In [86]:
DTC_good=DecisionTreeClassifier(criterion='gini',max_depth=6,min_samples_leaf=3,min_samples_split=9)
DTC_good.fit(Xtrain,Ytrain)
DTC_good.score(Xtest,Ytest)

0.8379888268156425

In [88]:
# 预测
test_pre_tree=DTC_good.predict(test)
test_pre_tree

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [100]:
len(test_pre_tree)

418